In [1]:
from __future__ import print_function, division
import numpy as np
from numpy import linalg as la
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import scipy.optimize as optimize
from PyAstronomy import pyasl

def generate_ellipse(num_pts, semi_major, eccentricity, inclination, periapsis, longitude, t_closest):
    m = 4.3 #*10**6 solar mass
    n = np.sqrt(m*4.300952282/3.08567758**2/semi_major**3)*3.155692661 # mean motion: M=nt=sqrt(G(M+m)/a**3)t
    mu = m*4.300952282#mpc*(km/s)**2
    t = t_start - t_closest
    error = 10**(-5)
    flat_ellipse = np.zeros((num_pts, 4))
    flat_velocity = np.zeros((num_pts, 4))
#------------------------------
    def eccentric_anomaly(eccentricity, m_anomaly, error):
        ks = pyasl.MarkleyKESolver()
        E = ks.getE(m_anomaly, eccentricity)
        return E
#------------------------------
    def position(semi_major, eccentricity, e_anomaly, t):
        z = 0.
        nu = 2*np.arctan2(np.sqrt(1+eccentricity)*np.sin(e_anomaly/2), np.sqrt(1-eccentricity)*np.cos(e_anomaly/2))
        y = semi_major*(1-eccentricity*np.cos(e_anomaly))*np.cos(nu)
        x = semi_major*(1-eccentricity*np.cos(e_anomaly))*np.sin(nu)
        r = np.column_stack((x, y, z, t))#mas
        return r
#----------------------------------
    def velocity(mu, e_anomaly, semi_major, eccentricity, t):#(m/s)
        vy = -np.sin(e_anomaly)*np.sqrt(mu*semi_major)/semi_major/(1-eccentricity*np.cos(e_anomaly))
        vx = np.sqrt((1 - eccentricity**2)*mu*semi_major)*np.cos(e_anomaly)/semi_major/(1-eccentricity*np.cos(e_anomaly))
        vz = 0.
        v = np.column_stack((vx, vy, vz, t))
        return v
#-----------------------------
    # Create an ellipse in the xy plane
    for i in range(num_pts):
        m_anomaly = n*t
        e_anomaly = eccentric_anomaly(eccentricity, m_anomaly, error)
        flat_ellipse[i] = position(semi_major, eccentricity, e_anomaly, t+t_closest)
        flat_velocity[i] = velocity(mu, e_anomaly, semi_major, eccentricity, t+t_closest)
        t += dt
#----------------------------
    # Create a rotation matrix using Eulers angles
    rotation_matrix = np.array([[np.cos(-longitude)*np.cos(-inclination)*np.cos(-periapsis) - np.sin(-longitude)*np.sin(-periapsis), -np.cos(-periapsis)*np.sin(-longitude) - np.cos(-longitude)*np.cos(-inclination)*np.sin(-periapsis), np.cos(-longitude)*np.sin(-inclination)], [np.cos(-longitude)*np.sin(-periapsis) + np.cos(-inclination)*np.cos(-periapsis)*np.sin(-longitude), np.cos(-longitude)*np.cos(-periapsis) - np.cos(-inclination)*np.sin(-longitude)*np.sin(-periapsis), np.sin(-longitude)*np.sin(-inclination)], [-np.cos(-periapsis)*np.sin(-inclination), np.sin(-inclination)*np.sin(-periapsis), np.cos(-inclination)]])#ZYZ
#-----------------------------
    # Rotate the ellipse
    rotated_ellipse = np.zeros((num_pts, 7))
    for i in range(num_pts):
        rotated_r = np.dot(rotation_matrix, flat_ellipse[i, 0:3])
        rotated_v = np.dot(rotation_matrix, flat_velocity[i, 0:3])
        rotated_ellipse[i] = np.column_stack((rotated_r[0], rotated_r[1], rotated_r[2], rotated_v[0], rotated_v[1], rotated_v[2], flat_ellipse[i, 3]))
    return rotated_ellipse
#------------------------------------------------------------------------------------------------------

dec_dates = []
row_str = []

def inquiry_positions():

    num_pts = int((t_end-t_start)/dt)
    
#ellipse and orbit is generated and stored in an array, the 7th column stores each timestep
    dummy_ellipse = generate_ellipse(num_pts, semi_major, eccentricity, inclination, periapsis, longitude, t_closest)

# print (type(dummy_ellipse)) = <class 'numpy.ndarray'>

#=============================================================
#now the code to search investigated dates and print respective offsets

#import a text file containing all the dates needed
    dates_file = open("orbit_plot_dates.txt", 'r')

    for each_line in dates_file.readlines():
        
    #rounding the decimal dates to 3 decimal places just in case it wasn't already
        dates_rounded = round(float(each_line), 3)
        dec_dates.append(dates_rounded)
    
    print("Imported dates: ")
    print(dec_dates)
    
    #=============================================================
    
#Preparing an output file

    output_file = open("inquired offsets.txt", 'w+')
    
    #Printing useful information as headers
    orbit_parameter = semi_major, eccentricity, inclination, periapsis, longitude, t_closest
    generated_length = t_start, t_end , dt
    
    #orbit_parameter & generated_length are "tuples", convert to string for ".write" function
    orbit_parameter_str = str(orbit_parameter).replace("(","").replace(")","")
    generated_length_str = str(generated_length).replace("(","").replace(")","")
    
    print ("Orbital parameter given: " + orbit_parameter_str)
    print ("Starting, Ending Year & Timestep: " + generated_length_str)
        
    output_header = "Orbital Parameters Given:\n" + "semi_major, eccentricity, inclination, periapsis, longitude, t_closest:\n" + orbit_parameter_str + "\n\nstarting year, ending year, timestep:\n" + generated_length_str + "\n\n"
    output_file.write(output_header)
    
    #using string formatting to print the parameters in another format
    #orbit_parameter_alt = ("semi major axis: %10.5f, eccentricity: %10.5f, inclination: %10.5f, periapsis: %10.5f, longitude: %10.5f, time of closest approach: %10.5f"%(semi_major, eccentricity, inclination, periapsis, longitude, t_closest))
    #output_file.write(orbit_parameter_alt)
    
    output_body = "Here are the Dates & offsets from Sag A* inquried for this orbit:\nDates\tRA (marcsec)\tDEC (marcsec)\n"
    output_file.write(output_body)
    
    #=============================================================
    
#for each specified dates at 7th column, print the entire row for positions 
#array indice - array[row, column]; and the first one is [0]

    for each_date in dec_dates:
        if t_start <= each_date <= t_end:
            row_index = (each_date-t_start)/dt
            
            # floating number operation is fucked up, row_index ends up with decimals even when it should be an integer
            # rounding is required, otherwise int() simply rounds down everything.
            row_index = int(round(row_index,0))
            
    #originally written to print the entire row:
            #row = dummy_ellipse[row_index,:]    #"row" type: numpy.ndarray
           
    #we found that dummy_ellipse[0][1] are RA, DEC in marcsec respectively, after divided by a factor of 40
    #we only need dummy_ellipse[0][1][6] column: RA, Dec offsets, Time
    
            date_out = round(dummy_ellipse[row_index,6],5)
            RA = float(dummy_ellipse[row_index,0])/40*1000 #marcsec
            Dec = float(dummy_ellipse[row_index,1])/40*1000 #marcsec
            
            #row_str.append(str(row_index))
            row_str.append(str(date_out))
            row_str.append(str(RA))
            row_str.append(str(Dec))         
                        
    #need to change back to strings for .write
            #for value in row:
            #    row_str.append(str(value))
                
            print(row_str)
            
            output_file.write('\t'.join(row_str))
            output_file.write("\n")
            row_str.clear()            
            
        else:
            print("Out of bounds!")
            output_file.write("Out of bounds!")
            output_file.write("\n")
        
    
    #print(np.where(a==2000.001)[0][0])
    #print(type(np.where(a==2000)))
    #b = (np.where(a==2000.001)[0][0])
    #print(dummy_ellipse[b,:])      
              
    
      #=============================================================   
        
    #output_file.write('\n'.join(list3))
    
    dates_file.close()   
    output_file.close()


#=============================================================   

#give the orbital parameters & time settings
t_start = 2000.
t_end = 2300.
dt = 0.001
semi_major, eccentricity, inclination, periapsis, longitude, t_closest = 55.86967198269051 , 0.5054224605883859 , 1.5700713217191662 , 0.6826015724425499 , 1.2130054482171886 , 2076.7423109189544


inquiry_positions()


Imported dates: 
[2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0, 2022.0, 2023.0, 2024.0, 2025.0, 2026.0, 2027.0, 2028.0, 2029.0, 2030.0, 2031.0, 2032.0, 2033.0, 2034.0, 2035.0, 2036.0, 2037.0, 2038.0, 2039.0, 2040.0, 2041.0, 2042.0, 2043.0, 2044.0, 2045.0, 2046.0, 2047.0, 2048.0, 2049.0, 2050.0, 2051.0, 2052.0, 2053.0, 2054.0, 2055.0, 2056.0, 2057.0, 2058.0, 2059.0, 2060.0, 2061.0, 2062.0, 2063.0, 2064.0, 2065.0, 2066.0, 2067.0, 2068.0, 2069.0, 2070.0, 2071.0, 2072.0, 2073.0, 2074.0, 2075.0, 2076.0, 2077.0, 2078.0, 2079.0, 2080.0, 2081.0, 2082.0, 2083.0, 2084.0, 2085.0, 2086.0, 2087.0, 2088.0, 2089.0, 2090.0, 2091.0, 2092.0, 2093.0, 2094.0, 2095.0, 2096.0, 2097.0, 2098.0, 2099.0, 2100.0, 2101.0, 2102.0, 2103.0, 2104.0, 2105.0, 2106.0, 2107.0, 2108.0, 2109.0, 2110.0, 2111.0, 2112.0, 2113.0, 2114.0, 2115.0, 2116.0, 2117.0, 2118.0, 2119.0, 2120.0, 2121.0, 2122.0

In [3]:
# Convert Inquired Positions offsets from marcsec, to pixels & calculate expected Pixel coordinates (X, Y)
# Expected to read the format generated by "inquiry positions" script
# Need a reference table of Sgr A* pixel coordinate positions

#=============================================================
#=============================================================

# Part I: Converting RA/DEC offsets from marcsec to pixel unit

# import the "inquired positions" file & Generate 2 output results
inquired_positions = open("inquired offsets.txt", 'r')
output_offset_pixel_file = open("101_inquired_pos_for_orbital_plot.txt", 'w+')

# Temporary working lists
list_offsets_pix = []
list_offsets_pix_str = []

# Preparing an output file header

for i, mas_offsets in enumerate(inquired_positions.readlines()):
    if i < 7:
        print(mas_offsets)
        #output_offset_pixel_file.write(mas_offsets)
    
    if i == 8:
        output_title = "Here are the Dates & offsets from Sag A* inquried for this orbit:\nDates\tX-axis (pix)\tY-axis (pix)\n"
        print(output_title)
        #output_offset_pixel_file.write(output_title)
    
# starting at 10th line, where the imported offsets in marcsec are  
# entire line is read as one single string, need to split it into 3 values
# print(type(split_line)) --> split_line gives in  a list

    if i >= 9:
        split_mas_offsets = mas_offsets.replace('\n', '').split("\t")

# the dates do not need to be changed
        for j, index in enumerate(split_mas_offsets):
            if j == 0:
                list_offsets_pix.append(round(float(index), 3))    

# RA is inverted from X-axis                
            elif j == 1:
                number_pixel = round(float(index)/-13.3, 5)
                list_offsets_pix.append(number_pixel)
                
# DEC is not inverted             
            elif j == 2:
                number_pixel = round(float(index)/13.3, 5)
                list_offsets_pix.append(number_pixel)
        
        for output_offsets_pix in list_offsets_pix:
            list_offsets_pix_str.append(str(output_offsets_pix))

# write the result to output file
# Format: "Dates (3 decimals), X, Y (in pixel)"

        output_offset_pixel_file.write('\t'.join(list_offsets_pix_str))
        output_offset_pixel_file.write("\n")
        print(list_offsets_pix)
        
        list_offsets_pix.clear()
        list_offsets_pix_str.clear()
        
#=============================================================        
# close all files
inquired_positions.close()
output_offset_pixel_file.close()
        
#=============================================================
#=============================================================   



Orbital Parameters Given:

semi_major, eccentricity, inclination, periapsis, longitude, t_closest:

55.86967198269051, 0.5054224605883859, 1.5700713217191662, 0.6826015724425499, 1.2130054482171886, 2076.7423109189544



starting year, ending year, timestep:

2000.0, 2300.0, 0.001



Here are the Dates & offsets from Sag A* inquried for this orbit:
Dates	X-axis (pix)	Y-axis (pix)

[2000.0, -33.73667, 12.67785]
[2001.0, -34.45257, 12.94472]
[2002.0, -35.16237, 13.2093]
[2003.0, -35.86583, 13.4715]
[2004.0, -36.56268, 13.73122]
[2005.0, -37.25264, 13.98835]
[2006.0, -37.93543, 14.24278]
[2007.0, -38.61075, 14.49442]
[2008.0, -39.27831, 14.74314]
[2009.0, -39.93779, 14.98882]
[2010.0, -40.58887, 15.23136]
[2011.0, -41.23121, 15.47061]
[2012.0, -41.86448, 15.70646]
[2013.0, -42.48831, 15.93877]
[2014.0, -43.10234, 16.1674]
[2015.0, -43.70619, 16.39222]
[2016.0, -44.29947, 16.61307]
[2017.0, -44.88177, 16.8298]
[2018.0, -45.45267, 17.04226]
[2019.0, -46.01175, 17.25028]
[2020.0, -46.55857, 